In [43]:
import pandas as pd
import os

from typing import Dict, List

# Formatting Original Data

In [44]:
src_dir = os.getcwd()
data_dir = os.path.join(src_dir, "Data")
os.makedirs(data_dir, exist_ok=True)  # Make directory if it doesn't exist

## nba-players-stats/Seasons_Stats.csv

In [45]:
stat_50_17_file = os.path.join(src_dir, "nba-players-stats/Seasons_Stats.csv")
# Can do this because:
# https://pandas.pydata.org/pandas-docs/version/0.24/whatsnew/v0.24.0.html#optional-integer-na-support
cols_to_int = ['Year', 'Age', 'G', 'GS', 'MP', 'FG', 'FGA', '3P', '3PA', '2P', '2PA', 'FT', 'FTA',
              'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'] 
type_conv = dict.fromkeys(cols_to_int, 'Int64')
stat50_17 = pd.read_csv(stat_50_17_file, index_col=0, dtype=type_conv)

In [46]:
def format_stats_50_17(stats: pd.DataFrame, fill_dict: List) -> pd.DataFrame:
    """
    Formats 1950-2017 stats file. Note that this function modifies stats to save memory.
    In:
        type_conv: Dict containing column names to convert to int.
    Return: 
        DataFrame from 1990-2017 (season ends); modified in-place to save memory/
    """
    # Drop rows prior to 1990 (our salary data is from 1991 onwards)
    stats.drop(stats[stats["Year"] < 1990].index, inplace=True)
    # Drop NA columns; 
    stats.dropna(axis=1, how='all', inplace=True)
    # For players who had multiple teams for a year, keep the TOTAL row
    stats.drop_duplicates(subset=["Year", "Player"], inplace=True)
    # Fill NAs with 0s for certain columns
    stats.fillna(fill_dict, inplace=True)
    # Drop NA rows
    stats.dropna(inplace=True)  # Defaults: axis=0, how='any'
    # Remove asterisks from HOF players
    stats["Player"] = stats["Player"].str.rstrip('*')
    # Reset index
    stats.reset_index(drop=True, inplace=True) # Drop the old index (rather than adding it as a col)
    return stats

In [47]:
cols_to_fill = ['FG%', '3P%', '2P%', 'eFG%', 'FT%']
fill_dict = dict.fromkeys(cols_to_fill, 0.0)
stat90_17 = format_stats_50_17(stat50_17, fill_dict)
# Rename Year col to Season End for clarity
stat90_17.rename(columns={"Year": "Season End"}, inplace=True) 
# Show first and last 5 rows and columns
# iloc[row_idx_list, col_idx_list]
print(stat90_17.iloc[list(range(5)) + list(range(-5, 0)), 
                   list(range(5)) + list(range(-5, 0))])

       Season End             Player Pos  Age   Tm  STL  BLK  TOV   PF   PTS
0            1990         Mark Acres   C   27  ORL   36   25   70  248   362
1            1990      Michael Adams  PG   27  DEN  121    3  141  133  1221
2            1990       Mark Aguirre  SF   30  DET   34   19  121  201  1099
3            1990        Danny Ainge  PG   30  SAC  113   18  185  238  1342
4            1990        Mark Alarie  PF   26  WSB   60   39  101  219   860
12292        2017        Cody Zeller  PF   24  CHO   62   58   65  189   639
12293        2017       Tyler Zeller   C   27  BOS    7   21   20   61   178
12294        2017  Stephen Zimmerman   C   20  ORL    2    5    3   17    23
12295        2017        Paul Zipser  SF   22  CHI   15   16   40   78   240
12296        2017        Ivica Zubac   C   19  LAL   14   33   30   66   284


In [48]:
out_stats_name = "nba_stats_1990_2017.csv"
out_stats_path = os.path.join(data_dir, out_stats_name)

In [49]:
if not os.path.isfile(out_stats_path): stat90_17.to_csv(out_stats_path, index=False)

## Salaries Per Season (1990-2017)

In [35]:
sal91_18_file = os.path.join(src_dir, "NBA_Salary_History/NBA_Salary_History.csv")
# Don't care about team column
sal91_18 = pd.read_csv(sal91_18_file, usecols=["Season", "Player", "Salary"], 
                       na_values = ['Unknown'])

In [36]:
def format_sal(sal91_18: pd.DataFrame) -> pd.DataFrame:
    """
    Formats 1991-2018 salaries file. Note that this function modifies sal91_18 to save memory.
    In:
        sal91_18: DataFrame from 1991-2018 salaries file; modified in-place to save memory.
    Return:
        Formatted salaries DataFrame.
    """
    # Drop NA rows
    sal91_18.dropna(inplace=True)  # Defaults: axis=0, how='any'
    # Rename Season to Season Start
    sal91_18.rename(columns={"Season": "Season Start"}, inplace=True)
    # Strip -YY from Season Start years
    sal91_18['Season Start'] = sal91_18['Season Start'].str.replace(r'-(.*)$', '').astype(int)
    # Add Season End column; have to unpack the dictionary for assign
    sal91_18 = sal91_18.assign(**{'Season End': (lambda x: x['Season Start'] + 1)})
    sal91_18['Salary'] = sal91_18['Salary'].astype(int)
    return sal91_18

In [37]:
sal91_18 = format_sal(sal91_18)
print(sal91_18.head())
print(sal91_18.tail())

   Season Start           Player    Salary  Season End
0          2017    Kent Bazemore  16910113        2018
1          2017  Dennis Schroder  15500000        2018
2          2017    Miles Plumlee  12500000        2018
3          2017   Jamal Crawford  10942762        2018
4          2017  Marco Belinelli   6306060        2018
       Season Start            Player  Salary  Season End
13292          1990      Harvey Grant  475000        1991
13293          1990       Byron Irvin  375000        1991
13294          1990      A.J. English  275000        1991
13295          1990       Greg Foster  275000        1991
13296          1990  Haywoode Workman  120000        1991


In [38]:
sal91_18.dtypes

Season Start     int64
Player          object
Salary           int64
Season End       int64
dtype: object

In [39]:
out_sal_name = "nba_salaries_1991_2018.csv"
out_sal_path = os.path.join(data_dir, out_sal_name)

In [40]:
if not os.path.isfile(out_sal_path): sal91_18.to_csv(out_sal_path, index=False)

## Matching Salary Data to Player Data by Year

In [50]:
sal91_18_form_file = os.path.join(data_dir, "nba_salaries_1991_2018.csv")
sal91_18_form = pd.read_csv(sal91_18_form_file)
stat90_17_form_file = os.path.join(data_dir, "nba_stats_1990_2017.csv")
stat90_17_form = pd.read_csv(stat90_17_form_file)

In [51]:
merged = pd.merge(stat90_17_form, sal91_18_form[["Player", "Season Start", "Salary"]], 
                  how='inner', 
                  left_on=['Player', 'Season End'], 
                  right_on=['Player', 'Season Start'])
if "Season Start" in merged.columns: merged.pop("Season Start")

In [52]:
print(merged.iloc[list(range(5)) + list(range(-5, 0)), 
                   list(range(5)) + list(range(-5, 0))])

       Season End             Player Pos  Age   Tm  BLK  TOV   PF   PTS  \
0            1990         Mark Acres   C   27  ORL   25   70  248   362   
1            1990      Michael Adams  PG   27  DEN    3  141  133  1221   
2            1990       Mark Aguirre  SF   30  DET   19  121  201  1099   
3            1990        Danny Ainge  PG   30  SAC   18  185  238  1342   
4            1990        Mark Alarie  PF   26  WSB   39  101  219   860   
10639        2017        Cody Zeller  PF   24  CHO   58   65  189   639   
10640        2017       Tyler Zeller   C   27  BOS   21   20   61   178   
10641        2017  Stephen Zimmerman   C   20  ORL    5    3   17    23   
10642        2017        Paul Zipser  SF   22  CHI   16   40   78   240   
10643        2017        Ivica Zubac   C   19  LAL   33   30   66   284   

         Salary  
0        437000  
1        825000  
2       1115000  
3        725000  
4        500000  
10639  12584270  
10640   1709538  
10641   1312611  
10642   1312

In [53]:
merged.dtypes

Season End      int64
Player         object
Pos            object
Age             int64
Tm             object
G               int64
GS              int64
MP              int64
PER           float64
TS%           float64
3PAr          float64
FTr           float64
ORB%          float64
DRB%          float64
TRB%          float64
AST%          float64
STL%          float64
BLK%          float64
TOV%          float64
USG%          float64
OWS           float64
DWS           float64
WS            float64
WS/48         float64
OBPM          float64
DBPM          float64
BPM           float64
VORP          float64
FG              int64
FGA             int64
FG%           float64
3P              int64
3PA             int64
3P%           float64
2P              int64
2PA             int64
2P%           float64
eFG%          float64
FT              int64
FTA             int64
FT%           float64
ORB             int64
DRB             int64
TRB             int64
AST             int64
STL       

In [54]:
out_merged_name = "nba_stats_sal_merged_1990_2017.csv"
out_merged_path = os.path.join(data_dir, out_merged_name)

In [55]:
if not os.path.isfile(out_merged_path): merged.to_csv(out_merged_path, index=False)